## Hex GNN Experimentation

In [1]:
NUMAGENTS = 30
NUMEXPERIMENTS = 150  #10
TIME = 10 # seconds #25
VISUAL = False

In [2]:
import simulation.Visualizer
import simulation.World
import simulation.Agent
import pickle
from matplotlib import pyplot as plt

simulation.Agent.USE = "SIM"
simulation.Agent.CASE = "HEX"

for i in range(NUMEXPERIMENTS):
    if i%10== 0:
        print(str(i)+"/"+str(NUMEXPERIMENTS))
    filename = "./data/hex-logs/experiment-"+str(i)+"-log.p"
    with open(filename, "wb") as f:
        world = simulation.World.World(f)
        for j in range(int(NUMAGENTS)):
            a = simulation.Agent.Agent(swarm=j%2)
            world.agents.append(a)

        if VISUAL:
            v = simulation.Visualizer.Visualizer(world)
            v.run()
        else:
            while world.stepCount < TIME*10:
                world.step()
            world.closeWorld()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
0/150
10/150
20/150
30/150
40/150
50/150
60/150
70/150
80/150
90/150
100/150
110/150
120/150
130/150
140/150


In [35]:
import utils.HexDataset
import shutil

###############################
###  Gather and split data  ###
###############################

ta = 70 # Percentage used for training (remaining used for testing)

try:
    shutil.rmtree("/tmp/Hexs", ignore_errors=False, onerror=None)
except FileNotFoundError as e:
    pass
    
dataset = utils.HexDataset.HexDataset(root='/tmp/Hexs').shuffle()

train_data = dataset[0:int(ta/100.*len(dataset))]
test_data = dataset[int(ta/100.*len(dataset)):]

print(dataset[0])

Processing...


File: 0
File: 1
File: 2
File: 3
File: 4
File: 5
File: 6
File: 7
File: 8
File: 9
File: 10
File: 11
File: 12
File: 13
File: 14
File: 15
File: 16
File: 17
File: 18
File: 19
File: 20
File: 21
File: 22
File: 23
File: 24
File: 25
File: 26
File: 27
File: 28
File: 29
File: 30
File: 31
File: 32
File: 33
File: 34
File: 35
File: 36
File: 37
File: 38
File: 39
File: 40
File: 41
File: 42
File: 43
File: 44
File: 45
File: 46
File: 47
File: 48
File: 49
File: 50
File: 51
File: 52
File: 53
File: 54
File: 55
File: 56
File: 57
File: 58
File: 59
File: 60
File: 61
File: 62
File: 63
File: 64
File: 65
File: 66
File: 67
File: 68
File: 69
File: 70
File: 71
File: 72
File: 73
File: 74
File: 75
File: 76
File: 77
File: 78
File: 79
File: 80
File: 81
File: 82
File: 83
File: 84
File: 85
File: 86
File: 87
File: 88
File: 89
File: 90
File: 91
File: 92
File: 93
File: 94
File: 95
File: 96
File: 97
File: 98
File: 99
File: 100
File: 101
File: 102
File: 103
File: 104
File: 105
File: 106
File: 107
File: 108
File: 109
File: 110


Done!


In [36]:
from torch_geometric.data import Data, DataLoader
from utils.models import *
from tqdm import tqdm
import numpy as np
import torch
from time import sleep

np.random.seed(2022)

num_epochs = 200
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

trainLoader = DataLoader(train_data, batch_size=32, shuffle=True)
testLoader = DataLoader(test_data, batch_size=32, shuffle=True)

node_dim = train_data[0].x.shape[1]
edge_dim = 0
if train_data[0].edge_attr: # if edge attributes used
    edge_dim = train_data[0].edge_attr.shape[1]
out_dim = train_data[0].y.shape[1]

# model = GenericGNN(node_dim, edge_dim, out_dim)
# optimizer = torch.optim.Adam(model.parameters(), lr=4e-5, weight_decay=5e-7)

In [37]:
# Size of discriminator output (real or fake)
d_output_size = 1
# Size of last hidden layer in the discriminator
d_hidden_size = 32

# Generator hyperparams
d_input=60
# Size of latent vector to give to generator
# z_size = 100
# Size of discriminator output (generated image)
# g_output_size = 784
# Size of first hidden layer in the generator
g_hidden_size = 32

In [38]:
# for row in dataset:
#     print((row.y).shape)
#     break
torch.cuda.is_available()

True

In [39]:
# instantiate discriminator and generator
from utils.discriminator_model_GNN import *
from utils.generator_model import *

D = Discriminator(node_dim, edge_dim, d_output_size)
G = Generator(node_dim, edge_dim, out_dim)

# check that they are as you expect
# print(D)
# print()
# print(G)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
D.to(device)
G.to(device)

Generator()

In [40]:
dataset[90]

Data(x=[30, 2], edge_index=[2, 694], y=[30, 2])

In [41]:
from torch import nn, optim
from torch.autograd.variable import Variable
d_optimizer = optim.Adagrad(D.parameters(), lr=4e-3, weight_decay=5e-7)
g_optimizer = optim.Adagrad(G.parameters(), lr=4e-3, weight_decay=5e-7)

In [42]:
loss = nn.BCEWithLogitsLoss()

In [68]:
# def ones_target(size):
#     '''
#     Tensor containing ones, with shape = size
#     '''
#     data = Variable(torch.ones(1, 1))
#     return data


# def zeros_target(size):
#     '''
#     Tensor containing zeros, with shape = size
#     '''
#     data = Variable(torch.zeros(1, 1))
#     return data

In [43]:
def ones_target(size):
    return torch.ones(size,1).to(device)

def zeros_target(size):
    return torch.zeros(size,1).to(device)

In [44]:
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = D(real_data.x,real_data.edge_index,real_data.edge_attr)
    # Calculate error and backpropagate
#     error_real = loss(prediction_real, ones_target(N) )
    error_real=loss(prediction_real,ones_target(real_data.x.shape[0]))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = D(fake_data.x,fake_data.edge_index,real_data.edge_attr)
    # Calculate error and backpropagate
#     error_fake = loss(prediction_fake, zeros_target(N))
    error_fake=loss(prediction_fake, zeros_target(real_data.x.shape[0]))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_real + error_fake, prediction_real, prediction_fake

In [45]:
def train_generator(optimizer, fake_data):
#     N = fake_data.size(0)    # Reset gradients
    optimizer.zero_grad()    # Sample noise and generate fake data
    prediction = D(fake_data.x,fake_data.edge_index,fake_data.edge_attr)   # Calculate error and backpropagate
#     error = loss(prediction, ones_target(N))
    error=loss(prediction, ones_target(fake_data.x.shape[0]))
    error.backward()    # Update weights with gradients
    optimizer.step()    # Return error
    return error

In [46]:
train_data

HexDataset(10395)

In [47]:
test_data

HexDataset(4455)

In [74]:
# row.y.size(0)
# prediction_real


In [75]:
# fake_data = G(train_data[0].x,train_data[0].edge_index,train_data[0].edge_attr)
# fake_data.size(0)
# D(dataset[0].y)
# for row,row1 in (train_data,trainLoader):
    
#     print(row)
#     print(row1)
#     break
train_data[1]

Data(x=[30, 2], edge_index=[2, 868], y=[30, 2])

In [33]:
i=0
for row in trainLoader:
#     print(row)
    i+=1
print(i)
print(row.edge_index[1])

325
tensor([  2,   5,   6,  ..., 806, 807, 808])


In [77]:
# # d_error
    
#     fake_data = G(row.x,row.edge_index,row.edge_attr).detach()

In [78]:
# training_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True, num_workers=1)
# validation_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False, num_workers=1)

In [48]:
# Create logger instance
# from utils.log import Logger
# logger = Logger(model_name='GGN', data_name='Hex_exp')# Total number of epochs to train
from tqdm.notebook import tqdm
num_epochs = 200
# D.to(device)
# G.to(device)
d_error=0
g_error=0
for epoch in range(num_epochs):
    print("Epoch",epoch)
    for i,row in tqdm(enumerate(trainLoader)):
#         N = real_batch.size(0)        # 1. Train Discriminator
        row.to(device)
#         real_data = row.y        # Generate fake data and detach 
        # (so gradients are not calculated for generator)
        generated_y = G(row.x,row.edge_index,row.edge_attr).detach()        # Train D
#         fake_data=row.x+generated_y
        row_real=row
        row_real.x=torch.cat((row_real.x,row_real.y),0)
        row_generated=row
        row_generated.x=torch.cat((row_generated.x,generated_y),0)
        d_error, d_pred_real, d_pred_fake =train_discriminator(d_optimizer, row_real, row_generated)

        # 2. Train Generator        # Generate fake data
        generated_y = G(row.x,row.edge_index,row.edge_attr)        # Train G
        row_generated=row
        row_generated.x=torch.cat((row_generated.x,generated_y),0)
        g_error = train_generator(g_optimizer, row_generated)        # Log batch error
        
    print('\n'+"|D_error|"+str(d_error)+"|G_error|"+str(g_error))

Epoch 0


0it [00:00, ?it/s]


|D_error|tensor(1.3863, device='cuda:0', grad_fn=<AddBackward0>)|G_error|tensor(0.6647, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch 1


0it [00:00, ?it/s]


|D_error|tensor(1.3863, device='cuda:0', grad_fn=<AddBackward0>)|G_error|tensor(0.6566, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch 2


...


|D_error|tensor(1.3863, device='cuda:0', grad_fn=<AddBackward0>)|G_error|tensor(0.6013, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch 165


0it [00:00, ?it/s]


|D_error|tensor(1.3863, device='cuda:0', grad_fn=<AddBackward0>)|G_error|tensor(0.6012, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
Epoch 166


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [80]:
# # !pip install tensorboardX
# for i,row in tqdm(enumerate(testLoader)):
# #         N = real_batch.size(0)        # 1. Train Discriminator
#         row.to(device)
# #         real_data = row.y        # Generate fake data and detach 
#         # (so gradients are not calculated for generator)
#         generated_y = G(row.x,row.edge_index,row.edge_attr).detach()  
#         print(generated_y)
#         print(row.y)
#         break

In [20]:
#from util import *
from random import random
from copy import deepcopy as copy
from tqdm import tqdm
from time import sleep
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

plt.rcParams['figure.figsize'] = (12, 4)

lossFunction = nn.MSELoss()

def train(model, optimizer, dataset):
    
    model.to(device)
    model.train()
    total_loss = 0.0
    num_items = 1
        
    for row in dataset:
        optimizer.zero_grad()
        pred = model(row.x, 
                     row.edge_index, 
                     row.edge_attr)
        loss = lossFunction(row.y, pred)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        num_items += 1
    return total_loss/num_items

def evaluate(model, dataset):
    
    model.to(device)
    model.eval()
    
    total_loss = 0.0
    num_items = 0
    
    num_outs = dataset[0].y.shape[1]
    
    predHist = [[] for _ in range(num_outs)]
    actualHist = [[] for _ in range(num_outs)]    

    for row in dataset:
        pred = model(row.x, 
                 row.edge_index, 
                 row.edge_attr)
        loss = lossFunction(row.y, pred)

        for i in range(len(pred[0])):
            predHist[i].append(pred[0][i].item())
            actualHist[i].append(row.y[0][i].item())
            
        total_loss += loss.item()
        num_items += 1
        
    return total_loss/num_items, actualHist, predHist

NameError: name 'plt' is not defined

In [12]:
num_epochs = 200
epoch = 0
nbins  = 100
minmax = 0.5
bins  = [((i*(minmax*2))/nbins) - minmax for i in range(nbins+1)]
losses = []

plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 100

for epoch in tqdm(range(epoch, num_epochs)):
       
    train_loss      = train(   model, optimizer, train_data)
    test_loss, a, p = evaluate(model,            test_data)
    
    losses.append([train_loss, test_loss])
    
    print("{train: " + str(train_loss) +"}, {test: " + str(test_loss) + "}")
    
    plt.rcParams['figure.figsize'] = (5, 3)    
    fig, ax = plt.subplots(1,1)
    fig.suptitle("Testing dataset", fontsize=16)
    ax.hist(p[0], bins=bins, alpha=0.5, label="predicted")
    ax.hist(a[0], bins=bins, alpha=0.5, label="actual")
    ax.legend()
    plt.show()
    
    torch.save(model.state_dict(), './models/hex-models/'+str(epoch)+'.mod')
    sleep(0.5)

  0%|          | 0/200 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
plt.plot([i for i in range(len(losses))],[losses[i][0] for i in range(len(losses))])
plt.plot([i for i in range(len(losses))],[losses[i][1] for i in range(len(losses))])

In [ ]:
model.eval()

In [ ]:
import math

xs = 0.07
xe = 0.4

MULT = 1
TARGET_DIST = 0.15 
DIP = 1 

def lj(x, e):
    
    dist = math.sqrt(((x[1][0]-x[0][0])**2)+((x[1][1]-x[0][1])**2))
    epsilon = DIP*MULT
    sigma = (TARGET_DIST*MULT)/(2**(1/6))
    mag = min((epsilon*4) * (pow((sigma/dist),12)-pow((sigma/dist),6)), 10)
    
    angle = math.pi
    if mag > 0:
        angle = 0
    
    return mag, angle
    
    
numScatter = 5000
numPoints = 1000
delta = (1/numPoints)*(xe-xs)

plotDataX = []
plotDataPredY = []
plotDataPredA = []
plotDataPredX = []
plotDataActY = []
plotDataActA = []
plotDataEqnY = []


for i in range(numPoints):
    
    data = [[0, 0], [0, 0]]
    xVal = delta*i+delta + xs
    data[1] = [xVal, 0]
    plotDataX.append(xVal)
    
    e1 = [0, 1]
    e2 = [1, 0]
    
    x = torch.tensor(data, dtype=torch.float)
    e = torch.tensor([e1, e2], dtype=torch.long)
    
    actualMag, actualAng = lj(x, e)
    
    plotDataActY.append(actualMag)
    plotDataActA.append(math.degrees(actualAng))
    
pois = []

for i in range(numScatter):
    
    data = [[0, 0] for _ in range(20)]
    ox = random()
    oy = random()
    d = (random()*(xe-xs)+xs)
    a = random() * (2*math.pi)
    no = ox + d*math.cos(a)
    ny = oy + d*math.sin(a)
    
    data[0] = [ox, oy]
    data[1] = [no, ny]
    e1 = [0, 1]
    e2 = [1, 0]
    
    x = torch.tensor(data, dtype=torch.float)
    e = torch.tensor([e1, e2], dtype=torch.long)
    pred = model(x, e)
    ljMag, ljAng = lj(x, e)
    pois.append([ox, oy, no, ny, pred[0][0], pred[0][1], d])
    
    mag = math.sqrt((pred[0][0])**2 + (pred[0][1])**2)
    ang = abs(math.degrees(math.atan2(pred[0][1], pred[0][0]) - math.atan2(oy-ny, ox-no)))
    
    actAng  = math.degrees(ljAng)
       
    if ang < 90: # clip ang to 0
        mag = mag
    else: # clip ang to 180
        mag = -mag
    
    plotDataPredX.append(d)
    plotDataPredA.append(abs(actAng-ang))
    plotDataPredY.append(mag)
#    plotDataEqnY.append(eqnMag)

plt.rcParams['figure.figsize'] = [12, 3]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

fig, (ax1, ax2) = plt.subplots(1,2)

ys = -1.5* MULT
ye = 1.2*MULT

ax1.plot(plotDataX, plotDataActY, label='Actual Output from LJ Potential')
ax1.scatter(plotDataPredX, plotDataPredY, s=1, color=[1, 0, 0], alpha=0.1, label='Predicted by GNN of 20 agents')
ax1.set_xlim([xs, xe])
ax1.set_ylim([ys, ye])
ax1.plot([xs, xe], [0, 0], 'k--')
ax1.legend()
ax1.set_xlabel("Distance Between Two Agents (World Units)")
ax1.set_ylabel("Magnitude of Acceleration")

pickle.dump({'dist':plotDataPredX, 'mag':plotDataPredY}, open("GNNOutputData.p", 'wb'))

ax2.plot(plotDataX, plotDataActA, 'k--')
ax2.scatter(plotDataPredX, plotDataPredA, s=1, color=[1, 0, 0], alpha=0.1, label='Predicted by GNN of 20 agents')
ax2.set_xlim([xs, xe])
ax2.set_ylim([-5, 185])
ax2.plot([xs, xe], [0, 0], 'k--')
ax2.plot([xs, xe], [180, 180], 'k--')
ax2.set_xlabel("Distance Between Two Agents (World Units)")
ax2.set_ylabel("Angle Error of Acceleration (deg)")

ax1.set_title('Magnitude of Force (Hexagonal)')
ax2.set_title('Angle Error (Hexagonal)')     

plt.show()
